In [11]:
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

In [3]:
import time
import datetime

time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(time.time()))

'2019-03-23 21:59:21'

In [4]:
import pandas as pd

df = pd.read_csv('train.csv',index_col=0)

df.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
Id,,,,,,,,,,,,,,,,,,,,,
1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [5]:
df_desc=pd.DataFrame([],index=df.columns)
df_desc['Unique']=df.nunique()
df_desc['# Missing']=df.isna().sum()
df_desc['# Zeros']=[sum(df[col_nm]==0) for col_nm in df.columns]

df_desc['Available']=(len(df)-df_desc['# Missing']-df_desc['# Zeros'])
df_desc['% Available']=100*((df_desc['Available']/len(df)).round(2))
df_desc['Types']=df.dtypes

from random import choice, sample
df_desc['Sample']=[sample(set(df[col_nm].fillna('')),min(10,len(set(df[col_nm])))) for col_nm in df.columns]


pd.set_option('display.max_rows', 500)
df_desc

,Unique,# Missing,# Zeros,Available,% Available,Types,Sample
MSSubClass,15,0,0,1460,100.0,int64,"[20, 180, 50, 80, 75, 190, 90, 40, 30, 70]"
MSZoning,5,0,0,1460,100.0,object,"[RL, RM, FV, C (all), RH]"
LotFrontage,110,259,0,1201,82.0,float64,"[114.0, 152.0, 84.0, 79.0, 149.0, 118.0, 51.0,..."
LotArea,1073,0,0,1460,100.0,int64,"[11664, 23595, 12378, 4130, 5381, 9405, 7838, ..."
Street,2,0,0,1460,100.0,object,"[Pave, Grvl]"
Alley,2,1369,0,91,6.0,object,"[Grvl, , Pave]"
LotShape,4,0,0,1460,100.0,object,"[Reg, IR1, IR3, IR2]"
LandContour,4,0,0,1460,100.0,object,"[Lvl, HLS, Low, Bnk]"
Utilities,2,0,0,1460,100.0,object,"[NoSeWa, AllPub]"
LotConfig,5,0,0,1460,100.0,object,"[FR3, Inside, CulDSac, FR2, Corner]"


In [6]:
cats1=df_desc[df_desc['Types']=='object']
cats1_list=list(cats1.index)
len(cats1)

43

In [7]:
# sparse square foot features, square footage of rare features
num_sprs=['LowQualFinSF' , 'PoolArea', 'MiscVal', '3SsnPorch','MiscVal' ] 
# Ordinal values
ordinals=['OverallQual','OverallCond','BsmtFullBath','BsmtHalfBath','FullBath', 'HalfBath', 'BedroomAbvGr',
       'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces', 'GarageCars']
mo_yr=['MoSold','YrSold']

cats2=df_desc[(df_desc['Types']!='object')&(df_desc['Unique']<25)]
cats2_list=list(set(cats2.index)-set(mo_yr)-set(num_sprs))
len(cats2_list)

12

In [8]:
y_list=['SalePrice']

nums=df_desc[(df_desc['Types']!='object')&(df_desc['Unique']>=25)]

nums_list=list(nums.index)+num_sprs
nums_list=list(set(nums_list)-set(y_list))

len(nums_list)


22

In [9]:
#Adding back month and year sold 
df['DtSold']=df['YrSold']+df['MoSold']/12

nums_list=nums_list+['DtSold']

len(nums_list)


23

In [12]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

df_nums=df[nums_list].fillna(0)
scld_nums=scaler.fit_transform(df_nums)

df_nums=pd.DataFrame(scld_nums,columns=df_nums.columns).replace([np.inf, np.nan,-np.inf], 0)

df_nums.shape

(1460, 23)

In [13]:
df_nums.head()

,LotArea,WoodDeckSF,LotFrontage,YearRemodAdd,MasVnrArea,TotalBsmtSF,OpenPorchSF,BsmtFinSF1,ScreenPorch,3SsnPorch,...,EnclosedPorch,PoolArea,BsmtFinSF2,GarageYrBlt,MiscVal,GarageArea,GrLivArea,2ndFlrSF,LowQualFinSF,DtSold
0,0.033420,0.000000,0.207668,0.883333,0.12250,0.140098,0.111517,0.125089,0.0,0.0,...,0.000000,0.0,0.0,0.996517,0.0,0.386460,0.259231,0.413559,0.0,0.462963
1,0.038795,0.347725,0.255591,0.433333,0.00000,0.206547,0.000000,0.173281,0.0,0.0,...,0.000000,0.0,0.0,0.983085,0.0,0.324401,0.174830,0.000000,0.0,0.296296
2,0.046507,0.000000,0.217252,0.866667,0.10125,0.150573,0.076782,0.086109,0.0,0.0,...,0.000000,0.0,0.0,0.995522,0.0,0.428773,0.273549,0.419370,0.0,0.592593
3,0.038561,0.000000,0.191693,0.333333,0.00000,0.123732,0.063985,0.038271,0.0,0.0,...,0.492754,0.0,0.0,0.994030,0.0,0.452750,0.260550,0.366102,0.0,0.018519
4,0.060576,0.224037,0.268371,0.833333,0.21875,0.187398,0.153565,0.116052,0.0,0.0,...,0.000000,0.0,0.0,0.995025,0.0,0.589563,0.351168,0.509927,0.0,0.648148


In [14]:
df_mdl=pd.get_dummies(df[cats1_list+cats2_list])

df_mdl.shape

(1460, 264)

In [15]:
len(set(df_mdl.columns))

264

In [16]:
df_mdl[list(df_nums.columns)]=df_nums
X=df_mdl.copy()
X.shape

(1460, 287)

In [17]:
len(set(X.columns))

287

In [18]:
Y=df[y_list]
Y.shape

(1460, 1)

In [19]:
# NB some values are not changed to categorical even if using to get dummies as long as values are numerical
# Needs a manual fit to categorical

In [20]:
from sklearn.metrics import auc, accuracy_score, confusion_matrix, mean_squared_error
import xgboost as xgb

In [21]:
xgb_model = xgb.XGBRegressor(objective="reg:linear", random_state=42)

xgb_model.fit(X, Y)

y_pred = xgb_model.predict(X)

mse=mean_squared_error(Y, y_pred)

print(np.sqrt(mse))

20139.4094513


In [22]:
Y=np.log(np.array(df[y_list]))

In [23]:
xgb_model = xgb.XGBRegressor(objective="reg:linear", random_state=42)

xgb_model.fit(X, Y)

y_pred = xgb_model.predict(X)

mse=mean_squared_error(Y, y_pred)

print(np.sqrt(mse))

0.111559773279


# Test Set

In [24]:
df_test = pd.read_csv('test.csv',index_col=0)
df_test.shape

(1459, 79)

In [25]:
df_test_mdl=pd.get_dummies(df_test[cats1_list+cats2_list])

print(df_mdl.shape)
print(df_test_mdl.shape)

(1460, 287)
(1459, 246)


In [26]:
df_test_mdl=df_test_mdl.reindex(columns = df_mdl.columns, fill_value=0)
df_test_mdl.shape

(1459, 287)

In [27]:
any(df_test_mdl.columns==df_mdl.columns)

True

In [28]:
#Adding back month and year sold 
df_test['DtSold']=df_test['YrSold']+df_test['MoSold']/12

In [29]:
df_test_nums=df_test[nums_list].fillna(0)
scld_test_nums=scaler.transform(df_test_nums)

df_test_nums=pd.DataFrame(scld_test_nums,columns=df_test_nums.columns).replace([np.inf, np.nan,-np.inf], 0)

print(df_nums.shape)
print(df_test_nums.shape)

(1460, 23)
(1459, 23)


In [30]:
df_test_mdl[list(df_nums.columns)]=df_test_nums
X_test=df_test_mdl.copy()
X_test.shape

(1459, 287)

In [31]:
y_test_pred = xgb_model.predict(X_test)

In [38]:
submit_1=pd.DataFrame(X_test.index,columns=['Id'])
#submit_1['SalePrice']=np.exp(y_test_pred)
submit['SalePrice']=y_test_pred

submit=submit.set_index('Id')
submit.to_csv('submit_1.csv')
submit_1.head()

,SalePrice
Id,
1461,117994.929688
1462,148821.906250
1463,163419.968750
1464,196598.609375
1465,214596.265625


In [204]:
# 0.16 (pl 3000 on LB)
# ca 1hr distracted
time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(time.time()))

'2019-03-23 20:02:17'

# Next steps

- Cross validation/Hyperparamter tuning
- More models
- Ensemble model - where results are weighted from different models
    https://www.kaggle.com/itslek/blend-stack-lr-gb-0-10649-house-prices-v57
- Further transformation of variables
- Feature engineering

In [32]:
from sklearn.linear_model import ElasticNetCV, LassoCV, RidgeCV
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import mean_squared_error

from mlxtend.regressor import StackingCVRegressor

from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

In [48]:
X=X.replace([np.inf,np.nan],0)

kfolds = KFold(n_splits=10, shuffle=True, random_state=42)


# rmsle
def rmsle(y, y_pred):
    return np.sqrt(mean_squared_error(y, y_pred))


# build our model scoring function
def cv_rmse(model, X=X):
    rmse = np.sqrt(-cross_val_score(model, X, np.array(Y).reshape((len(Y))),
                                    scoring="neg_mean_squared_error",
                                    cv=kfolds))
    return (rmse)


# setup models    
alphas_alt = [14.5, 14.6, 14.7, 14.8, 14.9, 15, 15.1, 15.2, 15.3, 15.4, 15.5]
alphas2 = [5e-05, 0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008]
e_alphas = [0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007]
e_l1ratio = [0.8, 0.85, 0.9, 0.95, 0.99, 1]

In [35]:


ridge = make_pipeline(RobustScaler(),
                      RidgeCV(alphas=alphas_alt, cv=kfolds,))

lasso = make_pipeline(RobustScaler(),
                      LassoCV(max_iter=1e7, alphas=alphas2,
                              random_state=42, cv=kfolds))

elasticnet = make_pipeline(RobustScaler(),
                           ElasticNetCV(max_iter=1e7, alphas=e_alphas,
                                        cv=kfolds, random_state=42, l1_ratio=e_l1ratio))
                                        
svr = make_pipeline(RobustScaler(),
                      SVR(C= 20, epsilon= 0.008, gamma=0.0003,))


gbr = GradientBoostingRegressor(n_estimators=3000, learning_rate=0.05,
                                   max_depth=4, max_features='sqrt',
                                   min_samples_leaf=15, min_samples_split=10, 
                                   loss='huber', random_state =42)
                                   

lightgbm = LGBMRegressor(objective='regression', 
                                       num_leaves=4,
                                       learning_rate=0.01, 
                                       n_estimators=5000,
                                       max_bin=200, 
                                       bagging_fraction=0.75,
                                       bagging_freq=5, 
                                       bagging_seed=7,
                                       feature_fraction=0.2,
                                       feature_fraction_seed=7,
                                       verbose=-1,
                                       #min_data_in_leaf=2,
                                       #min_sum_hessian_in_leaf=11
                                       )
                                       

xgboost = XGBRegressor(learning_rate=0.01, n_estimators=3460,
                                     max_depth=3, min_child_weight=0,
                                     gamma=0, subsample=0.7,
                                     colsample_bytree=0.7,
                                     objective='reg:linear', nthread=-1,
                                     scale_pos_weight=1, seed=27,
                                     reg_alpha=0.00006, random_state=42)



In [36]:
# stack
stack_gen = StackingCVRegressor(regressors=(ridge, lasso, elasticnet,
                                            gbr, xgboost, lightgbm),
                                meta_regressor=xgboost,
                                use_features_in_secondary=True)
                                


In [42]:
from datetime import datetime
print(datetime.now())

datetime.datetime(2019, 3, 23, 22, 12, 19, 519976)

In [50]:
print('TEST score on CV')


score = cv_rmse(ridge)
print("Kernel Ridge score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()), datetime.now(), )

score = cv_rmse(lasso)
print("Lasso score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()), datetime.now(), )



TEST score on CV
Kernel Ridge score: 0.1408 (0.0135)
 2019-03-23 22:28:47.219015
Lasso score: 0.1390 (0.0144)
 2019-03-23 22:28:54.103503


In [49]:

score = cv_rmse(elasticnet)
print("ElasticNet score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()), datetime.now(), )

score = cv_rmse(svr)
print("SVR score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()), datetime.now(), )

score = cv_rmse(lightgbm)
print("Lightgbm score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()), datetime.now(), )

score = cv_rmse(gbr)
print("GradientBoosting score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()), datetime.now(), )

score = cv_rmse(xgboost)
print("Xgboost score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()), datetime.now(), )


ElasticNet score: 0.1390 (0.0143)
 2019-03-23 22:21:02.132686
SVR score: 0.1406 (0.0134)
 2019-03-23 22:21:09.724767
Lightgbm score: 0.1428 (0.0114)
 2019-03-23 22:21:28.636999
GradientBoosting score: 0.1436 (0.0141)
 2019-03-23 22:23:09.447286
Xgboost score: 0.1406 (0.0123)
 2019-03-23 22:28:26.086612


In [52]:


print('START Fit')
print(datetime.now(), 'StackingCVRegressor')
stack_gen_model = stack_gen.fit(np.array(X), np.array(Y).reshape((len(Y))))
print(datetime.now(), 'elasticnet')
elastic_model_full_data = elasticnet.fit(X, Y)
print(datetime.now(), 'lasso')
lasso_model_full_data = lasso.fit(X, Y)
print(datetime.now(), 'ridge')
ridge_model_full_data = ridge.fit(X, Y)
print(datetime.now(), 'svr')
svr_model_full_data = svr.fit(X, Y)
print(datetime.now(), 'GradientBoosting')
gbr_model_full_data = gbr.fit(X, Y)
print(datetime.now(), 'xgboost')
xgb_model_full_data = xgboost.fit(X, Y)
print(datetime.now(), 'lightgbm')
lgb_model_full_data = lightgbm.fit(X, Y)


START Fit
2019-03-23 22:29:46.992459 StackingCVRegressor
2019-03-23 22:35:06.534630 elasticnet


/Users/MD/anaconda/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:1094: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


2019-03-23 22:35:09.714564 lasso


/Users/MD/anaconda/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:1094: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


2019-03-23 22:35:10.451641 ridge
2019-03-23 22:35:12.938527 svr


/Users/MD/anaconda/lib/python3.6/site-packages/sklearn/utils/validation.py:547: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


2019-03-23 22:35:13.854870 GradientBoosting
2019-03-23 22:35:25.201531 xgboost
2019-03-23 22:36:00.177213 lightgbm


TypeError: Wrong type(ndarray) for label.
It should be list, numpy 1-D array or pandas Series

In [53]:
print(datetime.now(), 'lightgbm')
lgb_model_full_data = lightgbm.fit(X, np.array(Y).reshape((len(Y))))

2019-03-23 22:36:52.095987 lightgbm


In [77]:

def blend_models_predict(X):
    return ((0.1 * elastic_model_full_data.predict(X)) + \
            (0.05 * lasso_model_full_data.predict(X)) + \
            (0.1 * ridge_model_full_data.predict(X)).reshape((len(X))) + \
            (0.1 * svr_model_full_data.predict(X)) + \
            (0.1 * gbr_model_full_data.predict(X)) + \
            (0.15 * xgb_model_full_data.predict(X)) + \
            (0.1 * lgb_model_full_data.predict(X)) + \
            (0.3 * stack_gen_model.predict(np.array(X))))
            
print('RMSLE score on train data:')
print(rmsle(np.array(Y).reshape((len(Y))), blend_models_predict(X)))


RMSLE score on train data:
0.0789785161866


In [80]:
X_test=X_test.replace([np.inf,np.nan],0)

y_pred=blend_models_predict(X_test)
y_pred=stack_gen_model.predict(np.array(X_test))
submit=pd.DataFrame(X_test.index,columns=['Id'])
submit['SalePrice']=np.exp(y_pred)
#submit['SalePrice']=blend_models_predict(X_test)

submit=submit.set_index('Id')
submit.to_csv('submit_2.csv')

submit.head()

,SalePrice
Id,
1461,118780.992188
1462,159097.421875
1463,169246.593750
1464,193282.921875
1465,189048.031250


In [81]:
print('RMSLE score on train data:')
print(rmsle(np.exp(np.array(Y).reshape((len(Y)))), np.exp(blend_models_predict(X))))

RMSLE score on train data:
15868.2690979
